In [3]:
import ipdb
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import pandas as pd
import matplotlib.pylab as pylab
import cartopy
from utils import constants as cnst
from eod import msg
from utils import u_grid, u_interpolate as u_int, constants as cnst, u_arrays, u_darrays, u_met
import pandas as pd
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as feature
import cartopy.io.shapereader as shapereader
import pickle as pkl
import glob

coast = shapereader.natural_earth(resolution='110m',
                                  category='physical',
                                  name='coastline')

countries = shapereader.natural_earth(resolution='110m',
                                      category='cultural',
                                      name='admin_0_countries')

pylab.rcParams['figure.figsize'] = (50., 50.)
%matplotlib inline

In [2]:
cd ..

/home/ck/pythonWorkspace/proj_CEH


In [20]:
filetag = 'srfc'
era_hour = 12

In [24]:
node_days = pkl.load(open(cnst.network_data + 'data/SOMS/saves/node3_MCS_matchTable.p', "rb"))

In [27]:
node_days

,prcp,q925,q650,t925,shear,month,hour,year,day,area,tmin,rh,tcwv,lat,lon
2004-04-09 00:00:00,1.314025,15.785048,6.643116,21.424097,12.285557,4.0,17.0,2004.0,9.0,8300.0,-62.490688,91.961121,57.024544,5.414777,-12.851019
2004-04-09 00:00:00,3.044069,14.231025,7.485625,23.838037,11.814247,4.0,18.0,2004.0,9.0,61550.0,-73.552764,70.169304,54.101547,6.094725,9.750353
2004-04-09 00:00:00,7.707650,14.260949,5.994442,27.298914,8.651213,4.0,21.0,2004.0,9.0,11375.0,-69.812521,58.225094,44.268383,8.461831,-5.751067
2004-04-09 00:00:00,6.934962,14.963483,6.016386,24.061700,10.243200,4.0,21.0,2004.0,9.0,62025.0,-70.000000,73.759201,42.655621,8.505196,-8.391715
2004-04-09 00:00:00,2.307837,14.896322,6.482527,23.490076,12.325956,4.0,21.0,2004.0,9.0,8925.0,-64.000000,75.517776,48.805210,7.146411,-9.258221
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2015-06-18 00:00:00,19.034942,14.920573,3.688829,23.057520,19.081163,6.0,21.0,2015.0,18.0,34025.0,-73.305625,78.165268,46.607536,7.136541,-2.880156
2015-06-18 00:00:00,5.963536,11.226146,6.405720,29.315637,6.080323,6.0,21.0,2015.0,18.0,6900.0,-67.177335,38.841370,33.639725,6.944409,11.129733
2015-06-18 00:00:00,3.055319,13.792298,4.451300,29.365564,15.501959,6.0,21.0,2015.0,18.0,9025.0,-80.108107,49.712143,38.433941,12.605580,3.558771
2015-06-18 00:00:00,4.878322,11.804416,3.916238,32.660059,17.072052,6.0,21.0,2015.0,18.0,10275.0,-72.000000,35.324425,32.855759,14.856554,-2.160377


In [22]:
def save_node_clim(filetag, node, era_hour):
    
    nodestr = 'node'+str(node)
    node_days = pkl.load(open(cnst.network_data + 'data/SOMS/saves/'+nodestr+'_MCS_matchTable.p', "rb"))
    
    #### potentially reduce MCS box here
    region = (node_days.lat>5) & (node_days.lat<9) & (node_days.lon>-10) & (node_days.lon<10)
    node_days = node_days[region]
    
    node_days_era = []
    for nd in node_days.index:
        node_days_era.append(nd.replace(hour=era_hour))
    node_days_era = pd.Series(node_days_era).values
        
    if filetag == 'pl':
        fft = 'pressure_levels'
    else:
        fft = 'surface'
        
    filebase = cnst.elements_drive + '/Africa/WestAfrica/ERA5/hourly/' 
    filepath = filebase + fft+'/ERA5*'+filetag+'.nc'
    out = '/home/ck/DIR/cornkle/data/SOMS/saves/'+nodestr+'_ERA5_MCS_mean_5-9N_'+filetag+'.nc'
    outdev = '/home/ck/DIR/cornkle/data/SOMS/saves/'+nodestr+'_ERA5_MCS_stddev_5-9N'+filetag+'.nc'
    
    def means(da, node_days):
        da = da.sel(time=(da['time.hour']==era_hour))
        try:
            da = da.sel(level=[925,850,750,650,550,250])
        except:
            pass
        
        if (filetag == 'pl'):
            if not 'z' in da.data_vars:
                ipdb.set_trace()
        
        intersect_dates = np.intersect1d(node_days, da.time.values)
        if len(intersect_dates) == 0:
            return None
        try:
            da = da.sel(time=intersect_dates)
        except:
            return None
        #ipdb.set_trace()
        da = u_darrays.flip_lat(da)
        return da
    
    srfc_list = []
    
    for f in glob.glob(filepath):
        print('Doing', f)
        srfc = xr.open_dataset(f) 
        srfc_concat = means(srfc,node_days_era)
        if srfc_concat is None:
            continue
        #ipdb.set_trace()
        srfc_list.append(srfc_concat)
    
    full = xr.concat(srfc_list, dim='time')
    #ipdb.set_trace()
    mean = full.mean('time')
    dev = full.std('time')
    
    mean.to_netcdf(out)
    dev.to_netcdf(outdev)

In [23]:
for ni in range(1,13):
#     nodestr='node'+str(1)
#     all_node_file = pkl.load(open(cnst.network_data + 'data/SOMS/saves/node_days.p', "rb"))
#     node_days = all_node_file[nodestr]
#     node_days_era = []
#     for nd in node_days:
#         node_days_era.append(nd.replace(hour=era_hour))
        
    save_node_clim(filetag, ni, era_hour)

Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/surface/ERA5_1979_01_srfc.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/surface/ERA5_1979_02_srfc.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/surface/ERA5_1979_03_srfc.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/surface/ERA5_1979_04_srfc.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/surface/ERA5_1979_05_srfc.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/surface/ERA5_1979_06_srfc.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/surface/ERA5_1979_07_srfc.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/surface/ERA5_1979_08_srfc.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/surface/ERA5_1979_09_srfc.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/surface/ERA5_1979_10_srfc.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/surface/ERA5_1979_11_srfc.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/surface/E

Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/surface/ERA5_1983_05_srfc.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/surface/ERA5_1983_06_srfc.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/surface/ERA5_1983_07_srfc.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/surface/ERA5_1983_08_srfc.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/surface/ERA5_1983_09_srfc.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/surface/ERA5_1983_10_srfc.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/surface/ERA5_1983_11_srfc.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/surface/ERA5_1983_12_srfc.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/surface/ERA5_1984_01_srfc.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/surface/ERA5_1984_02_srfc.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/surface/ERA5_1984_03_srfc.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/surface/E

Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/surface/ERA5_2002_07_srfc.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/surface/ERA5_2002_08_srfc.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/surface/ERA5_2002_09_srfc.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/surface/ERA5_2002_10_srfc.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/surface/ERA5_1993_02_srfc.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/surface/ERA5_1993_03_srfc.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/surface/ERA5_1993_04_srfc.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/surface/ERA5_1993_05_srfc.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/surface/ERA5_1993_06_srfc.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/surface/ERA5_1993_07_srfc.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/surface/ERA5_1993_08_srfc.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/surface/E

Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/surface/ERA5_1981_06_srfc.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/surface/ERA5_1981_07_srfc.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/surface/ERA5_1981_08_srfc.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/surface/ERA5_1981_09_srfc.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/surface/ERA5_1981_10_srfc.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/surface/ERA5_1981_11_srfc.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/surface/ERA5_1981_12_srfc.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/surface/ERA5_1982_01_srfc.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/surface/ERA5_1982_02_srfc.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/surface/ERA5_1982_03_srfc.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/surface/ERA5_1982_04_srfc.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/surface/E

Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/surface/ERA5_1989_02_srfc.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/surface/ERA5_1989_03_srfc.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/surface/ERA5_1989_04_srfc.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/surface/ERA5_1989_05_srfc.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/surface/ERA5_1989_06_srfc.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/surface/ERA5_1989_07_srfc.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/surface/ERA5_1989_08_srfc.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/surface/ERA5_1989_09_srfc.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/surface/ERA5_1989_10_srfc.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/surface/ERA5_1989_11_srfc.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/surface/ERA5_1989_12_srfc.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/surface/E

...

Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/surface/ERA5_2016_03_srfc.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/surface/ERA5_2016_04_srfc.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/surface/ERA5_2016_05_srfc.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/surface/ERA5_2016_06_srfc.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/surface/ERA5_2016_07_srfc.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/surface/ERA5_2016_08_srfc.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/surface/ERA5_2016_09_srfc.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/surface/ERA5_2016_10_srfc.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/surface/ERA5_2016_11_srfc.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/surface/ERA5_2016_12_srfc.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/surface/ERA5_2017_01_srfc.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/surface/E

Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/surface/ERA5_2019_02_srfc.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/surface/ERA5_2019_03_srfc.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/surface/ERA5_2019_04_srfc.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/surface/ERA5_2019_05_srfc.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/surface/ERA5_2013_01_srfc.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/surface/ERA5_2013_02_srfc.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/surface/ERA5_2013_03_srfc.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/surface/ERA5_2013_04_srfc.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/surface/ERA5_2013_05_srfc.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/surface/ERA5_2013_06_srfc.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/surface/ERA5_2013_07_srfc.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/surface/E

Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/surface/ERA5_1999_03_srfc.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/surface/ERA5_1999_04_srfc.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/surface/ERA5_1999_05_srfc.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/surface/ERA5_1999_06_srfc.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/surface/ERA5_1999_07_srfc.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/surface/ERA5_1999_08_srfc.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/surface/ERA5_1999_09_srfc.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/surface/ERA5_1999_10_srfc.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/surface/ERA5_1999_11_srfc.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/surface/ERA5_1999_12_srfc.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/surface/ERA5_2000_01_srfc.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/surface/E

Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/surface/ERA5_2011_07_srfc.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/surface/ERA5_2011_08_srfc.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/surface/ERA5_2011_09_srfc.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/surface/ERA5_2011_10_srfc.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/surface/ERA5_2011_11_srfc.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/surface/ERA5_2011_12_srfc.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/surface/ERA5_2012_01_srfc.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/surface/ERA5_2012_02_srfc.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/surface/ERA5_2012_03_srfc.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/surface/ERA5_2012_04_srfc.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/surface/ERA5_2012_05_srfc.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/surface/E

FileNotFoundError: [Errno 2] No such file or directory: '/home/ck/DIR/cornkle/data/SOMS/saves/node10_MCS_matchTable.p'

In [ ]:
'z' in da